In [1]:
import sys
!{sys.executable} -m pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import cv2
import numpy as np
import csv
import os
from tqdm import tqdm

def feature_tracking(prev_img, curr_img, prev_points):
    lk_params = dict(winSize=(21, 21), maxLevel=3,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    curr_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_img, curr_img, prev_points, None, **lk_params)
    
    if curr_points is None or status is None:
        return None, None
    
    good_old = prev_points[status.flatten() == 1]
    good_new = curr_points[status.flatten() == 1]
    
    return good_old, good_new

def detect_features(image, mask):
    feature_params = dict(maxCorners=200, qualityLevel=0.01, minDistance=30, blockSize=7)
    points = cv2.goodFeaturesToTrack(image, mask=mask, **feature_params)
    return points

def process_video(video_path, mask_path=None, results_folder='results'):
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    csv_file_path = os.path.join(results_folder, f"{video_name}.csv")
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video {video_path}")
        return
    
    ret, prev_frame = cap.read()
    if not ret or prev_frame is None:
        print("Cannot read the first frame")
        return
    
    frame_height, frame_width = prev_frame.shape[:2]
    
    mask = None
    if mask_path and os.path.exists(mask_path):
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None or mask.shape != (frame_height, frame_width):
            print("Mask image not found or dimensions do not match the video frame. No mask will be used.")
            mask = None
        else:
            print(f"Mask loaded from {mask_path}")
    
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_points = detect_features(prev_frame_gray, mask)
    
    if prev_points is None:
        print("No initial features detected. Exiting.")
        return
    
    trajectory = np.zeros((3, 1))
    poses = [trajectory]
    
    frame_num = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['frame_num', 'x', 'y', 'z'])
        
        with tqdm(total=total_frames, desc=f"Processing {video_name}") as pbar:
            while True:
                ret, frame = cap.read()
                if not ret or frame is None:
                    break
                
                curr_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
                prev_points, curr_points = feature_tracking(prev_frame_gray, curr_frame_gray, prev_points)
                
                if prev_points is None or curr_points is None or len(prev_points) < 5 or len(curr_points) < 5:
                    print("Not enough points to compute the essential matrix, re-detecting features")
                    prev_points = detect_features(prev_frame_gray, mask)
                    curr_points = detect_features(curr_frame_gray, mask)
                    if prev_points is None or curr_points is None or len(prev_points) < 5 or len(curr_points) < 5:
                        print("Not enough points detected, repeating last trajectory")
                        x, y, z = trajectory.flatten()
                        writer.writerow([frame_num, x, y, z])
                        prev_frame_gray = curr_frame_gray.copy()
                        frame_num += 1
                        pbar.update(1)
                        continue
                
                try:
                    prev_points = prev_points.reshape(-1, 2)
                    curr_points = curr_points.reshape(-1, 2)
                    prev_points = prev_points.astype(np.float32)
                    curr_points = curr_points.astype(np.float32)
                    
                    E, mask_e = cv2.findEssentialMat(curr_points, prev_points, focal=1.0, pp=(0.0, 0.0), method=cv2.RANSAC, prob=0.999, threshold=1.0)
                    
                    if E is None or E.shape != (3, 3):
                        print("Invalid essential matrix, repeating last trajectory")
                        x, y, z = trajectory.flatten()
                        writer.writerow([frame_num, x, y, z])
                        prev_frame_gray = curr_frame_gray.copy()
                        frame_num += 1
                        pbar.update(1)
                        continue
                    
                    _, R, t, _ = cv2.recoverPose(E, curr_points, prev_points, focal=1.0, pp=(0.0, 0.0))
                    
                    trajectory = trajectory + R @ t
                    poses.append(trajectory)
                    
                    x, y, z = trajectory.flatten()
                    writer.writerow([frame_num, x, y, z])
                    
                    prev_frame_gray = curr_frame_gray.copy()
                    prev_points = curr_points.reshape(-1, 1, 2)
                    
                    frame_num += 1
                except cv2.error as e:
                    print(f"Error processing frame: {e}, repeating last trajectory")
                    x, y, z = trajectory.flatten()
                    writer.writerow([frame_num, x, y, z])
                
                pbar.update(1)
    
    cap.release()

def process_videos(root_folder='own', mask_path=None, results_folder='own/cam'):
    if not os.path.exists(results_folder):
        os.makedirs(results_folder)
    
    videos = [os.path.join(root_folder, v) for v in os.listdir(root_folder) if v.endswith(('.mp4'))]
    
    for video in videos:
        process_video(video, mask_path, results_folder)

# Example usage
process_videos(mask_path='mask.png')


Mask loaded from mask.png


Processing 07bigroundzeroshot:   7%|▋         | 994/14946 [00:06<01:28, 158.11it/s]

Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Not enough points to compute the essential matrix, re-detecting features


Processing 07bigroundzeroshot:   9%|▊         | 1307/14946 [00:08<01:10, 193.19it/s]

Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Not enough points to compute the essential matrix, re-detecting features


...

Processing 06home2uni:  96%|█████████▌| 27580/28868 [03:01<00:06, 206.35it/s]

Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid esse

Processing 06home2uni:  96%|█████████▌| 27602/28868 [03:02<00:06, 208.50it/s]

Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid esse

Processing 06home2uni: 100%|█████████▉| 28867/28868 [03:08<00:00, 153.08it/s]
